# Import Libraries

In [32]:
import pandas as pd
import numpy as np
import urllib
import os
import sagemaker
import boto3
from sagemaker.session import s3_input, Session
from sagemaker.estimator import Estimator
from sagemaker.serializers import CSVSerializer

## Creating S3 Bucket

In [7]:
bucket_name = 'bankapplication-ardiansyahf'
my_region = boto3.session.Session().region_name

ap-southeast-2


In [8]:
s3 = boto3.resource('s3')

try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': my_region}
        )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error:', e)

S3 bucket created successfully


## Mapping path of the model in S3

In [10]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplication-ardiansyahf/xgboost-as-a-built-in-algo/output


## Downloading the dataset and storing in S3

In [12]:
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv')
except Exception as e:
    print('Data load error: ', e)

try:
    model_data = pd.read_csv('./bank_clean.csv', index_col=0)
    print('Success: Data loaded into dataframe')
except Exception as e:
    print('Data load error: ', e)

Success: downloaded bank_clean.csv
Success: Data loaded into dataframe


## Saving train test split into Buckets

In [14]:
# Train test split
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [15]:
# Saving train data
pd.concat([train_data['y_yes'], 
           train_data.drop(['y_no', 'y_yes'], axis=1)],
           axis=1).to_csv('train.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

[06/03/25 14:01:04] INFO     Found credentials from IAM Role:                                   ]8;id=68894;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=88621;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [16]:
# Saving test data
pd.concat([test_data['y_yes'], 
           test_data.drop(['y_no', 'y_yes'], axis=1)],
           axis=1).to_csv('test.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

[06/03/25 14:05:33] INFO     Found credentials from IAM Role:                                   ]8;id=943650;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=763279;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

# Building Models Xgboot-Inbuilt Algorithm

In [24]:
# XGBoost image URI and builds an XGBoost container.
region = sagemaker.Session().boto_region_name
image_uri = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=region,
    version='1.0-1'  # or your desired version
)

# initialize hyperparameters (from local environtment)
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

[06/03/25 14:51:43] INFO     Defaulting to only available Python version: py3                     ]8;id=704283;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=683697;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#610\610]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=337123;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=287825;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#534\534]8;;\

In [26]:
xgb = Estimator(
    image_uri=image_uri,
    role=role,
    instance_count=1,
    instance_type='ml.m5.2xlarge',
    volume_size=5,
    output_path=output_path,
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters,
    use_spot_instances=True,
    max_run=300,
    max_wait=600
)

In [28]:
xgb.fit({
    'train': s3_input_train,
    'validation': s3_input_test
})

[06/03/25 15:07:11] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=146551;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=867506;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=69159;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=803483;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-06-03-15-07-11-196                                             

2025-06-03 15:07:11 Starting - Starting the training job.........
2025-06-03 15:08:31 Starting - Preparing the instances for training...
2025-06-03 15:09:07 Downloading - Downloading the training image...
2025-06-03 15:09:43 Training - Training image download completed. Training in progress....
2025-06-03 15:10:16 Uploading - Uploading generated training model
2025-06-03 15:10:16 Completed - Training job completed
[2025-06-03 15:09:59.754 ip-10-0-185-115.ap-southeast-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CS

# Deploy Machine Learning Model As Endpoints


In [29]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

[06/03/25 15:11:25] INFO     Creating model with name: sagemaker-xgboost-2025-06-03-15-11-25-488    ]8;id=43558;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=848820;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[06/03/25 15:11:26] INFO     Creating endpoint-config with name                                     ]8;id=190049;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=201257;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\
                             sagemaker-xgboost-2025-06-03-15-11-25-488                                             

                    INFO     Creating endpoint with name sagemaker-xgboost-2025-06-03-15-11-25-488  ]8;id=562060;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=718836;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

------!

## Prediction on the test data

In [36]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [37]:
# Confussion Matrix

cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



# Deleting The Endpoints

In [38]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[06/03/25 15:37:04] WARNING  The endpoint attribute has been renamed in sagemaker>=2.            ]8;id=342979;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=722109;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Deleting endpoint with name: sagemaker-xgboost-2025-06-03-15-11-25-488 ]8;id=89061;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=181855;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4985\4985]8;;\

[{'ResponseMetadata': {'RequestId': 'ECTCFMCDVTE61WFV',
   'HostId': 'ZHCdh8dhX3bkSS2/sbc5TckU/yWD66cBT4wqcD9XDeli9GunMOL+PPTTcEZozPMDNo9JDInQ1Io=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'ZHCdh8dhX3bkSS2/sbc5TckU/yWD66cBT4wqcD9XDeli9GunMOL+PPTTcEZozPMDNo9JDInQ1Io=',
    'x-amz-request-id': 'ECTCFMCDVTE61WFV',
    'date': 'Tue, 03 Jun 2025 15:37:06 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-06-03-15-07-11-196/debug-output/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-06-03-15-07-11-196/profiler-output/system/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-06-03-15-07-11-196/debug-output/index/000000000/000000000000_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemake